In [1]:
import configparser
import sys
import os
from snowflake.snowpark import Session
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from snowflake.snowpark.functions import col, sproc,udf
from snowflake.snowpark.types import StringType, Variant, ArrayType
import seaborn as sns
import snowflake

In [2]:

def makeSnowflakeConnection(connection_name):
    global config
    config = configparser.RawConfigParser()
    config.read('configConnection.properties')
    warehouse = config.get(connection_name, 'warehouse')
    user = config.get(connection_name, 'user')
    password =config.get(connection_name, 'password')
    account=config.get(connection_name, 'account')
    role=config.get(connection_name, 'role')
    conn = dict(
                user=user,
                password=password,
                account=account,
                warehouse=warehouse,
                role=role
                )
    session = Session.builder.configs(conn).create()
    return session


session=makeSnowflakeConnection('DBMS_PROJECT')

In [3]:
session.use_database('COVID19')
session.use_schema('TRANSFORMED_DATA')

In [6]:
df_tweets=session.table('TWEETS_ALL_ML').to_pandas()
df_retweets=session.table('RETWEETS_ALL_ML').to_pandas()
df_users=session.table('USER_TABLE_CLEAN').to_pandas()

In [7]:
df_tweets.head()

,CREATED_AT,ID,TEXT,SOURCE,TRUNCATED,IN_REPLY_TO_STATUS_ID,IN_REPLY_TO_USER_ID,IN_REPLY_TO_SCREEN_NAME,USER_ID,USER_NAME,...,ENTITIES_SYMBOLS,ENTITIES_USER_MENTIONS,ENTITIES_URLS,LANG,EXTENDED_TWEET_FULL_TEXT,EXTENDED_ENTITIES_MEDIA,translated_text,sentiment,emotion,TOPIC_MODELS
0,Sun Apr 12 18:27:32 +0000 2020,1249403796872138757,@hanumanbeniwal सभी #AIIMS में नए भर्ती नियमों...,"<a href=""http://twitter.com/download/android"" ...",True,None,219188946,hanumanbeniwal,1020013084130271232,J. R. Huda,...,[],"[\n {\n ""id"": 219188946,\n ""id_str"": ""2...","[\n {\n ""display_url"": ""twitter.com/i/web/...",hi,@hanumanbeniwal सभी #AIIMS में नए भर्ती नियमों...,None,@hanumanbeniwal has been done in all #AIIMS ne...,Negative,anger,"Topic 3: doria (0.09), story (0.05), coronavir..."
1,Sun Apr 12 18:27:41 +0000 2020,1249403835711393792,@mediavakker @vrtnws En waarom zouden deze bur...,"<a href=""http://twitter.com/download/android"" ...",True,1249398547608469511,950489665513062401,mediavakker,54590065,Lynckx,...,[],"[\n {\n ""id"": 950489665513062401,\n ""id...","[\n {\n ""display_url"": ""twitter.com/i/web/...",nl,@mediavakker @vrtnws En waarom zouden deze bur...,None,@Mediafakker @vrtnws and why would these citiz...,Negative,anger,"Topic 0: would (0.04), nt (0.04), measures (0...."
2,Sun Apr 12 18:27:54 +0000 2020,1249403887871823872,"@_________xxz Mit den Worten ""digital"", ""innov...","<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1249401882596040705,909103079852822533,_________xxz,1104859974654668800,David,...,[],"[\n {\n ""id"": 909103079852822533,\n ""id...","[\n {\n ""display_url"": ""twitter.com/i/web/...",de,"@_________xxz Mit den Worten ""digital"", ""innov...",None,"@_________xxz with the words ""digital"", ""innov...",Negative,anger,"Topic 4: ucj4afnh8m (0.04), innovative (0.04),..."
3,Sun Apr 12 18:28:06 +0000 2020,1249403937746227200,Business Game Planning During Corona Virus - h...,"<a href=""http://www.mailchimp.com"" rel=""nofoll...",False,None,None,None,290265283,thepulsesd,...,[],[],"[\n {\n ""display_url"": ""mailchi.mp/6fce013...",en,None,None,Business Game Planning During Corona Virus - h...,Neutral,fear,"Topic 1: virus (0.06), wah (0.04), china (0.02..."
4,Sun Apr 12 18:28:17 +0000 2020,1249403986588971012,@HeatingBarrett @Grundfos_UK @gasappuk @CosyWa...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,1249285715529261057,3652922657,HeatingBarrett,1319972514,Dmc & Son Heating,...,[],"[\n {\n ""id"": 3652922657,\n ""id_str"": ""...",[],en,None,None,@HeatingBarrett @Grundfos_UK @gasappuk @CosyWa...,Positive,anger,"Topic 4: ucj4afnh8m (0.04), innovative (0.04),..."


In [8]:
print(df_tweets.columns.tolist())

['CREATED_AT', 'ID', 'TEXT', 'SOURCE', 'TRUNCATED', 'IN_REPLY_TO_STATUS_ID', 'IN_REPLY_TO_USER_ID', 'IN_REPLY_TO_SCREEN_NAME', 'USER_ID', 'USER_NAME', 'USER_SCREEN_NAME', 'USER_LOCATION', 'USER_DESCRIPTION', 'USER_PROTECTED', 'USER_FOLLOWERS_COUNT', 'USER_FRIENDS_COUNT', 'USER_LISTED_COUNT', 'USER_FAVOURITES_COUNT', 'USER_STATUSES_COUNT', 'USER_CREATED_AT', 'USER_GEO_ENABLED', 'GEO', 'COORDINATES', 'PLACE', 'IS_QUOTE_STATUS', 'QUOTE_COUNT', 'REPLY_COUNT', 'RETWEET_COUNT', 'FAVORITE_COUNT', 'ENTITIES_HASHTAGS', 'ENTITIES_SYMBOLS', 'ENTITIES_USER_MENTIONS', 'ENTITIES_URLS', 'LANG', 'EXTENDED_TWEET_FULL_TEXT', 'EXTENDED_ENTITIES_MEDIA', 'translated_text', 'sentiment', 'emotion', 'TOPIC_MODELS']


In [9]:
selected_columns_tweets = [
    'CREATED_AT',
    'ID',
    'TEXT',
    'IN_REPLY_TO_STATUS_ID',
    'IN_REPLY_TO_USER_ID',
    'USER_ID',
    'ENTITIES_HASHTAGS',
    'ENTITIES_USER_MENTIONS',
    'LANG',
    'translated_text',
    'sentiment',
    'emotion',
    'TOPIC_MODELS'
]

# Selecting the columns
df_tweets2 = df_tweets[selected_columns_tweets]

# Display the first few rows of the new DataFrame with the selected columns
df_tweets2.head()

,CREATED_AT,ID,TEXT,IN_REPLY_TO_STATUS_ID,IN_REPLY_TO_USER_ID,USER_ID,ENTITIES_HASHTAGS,ENTITIES_USER_MENTIONS,LANG,translated_text,sentiment,emotion,TOPIC_MODELS
0,Sun Apr 12 18:27:32 +0000 2020,1249403796872138757,@hanumanbeniwal सभी #AIIMS में नए भर्ती नियमों...,None,219188946,1020013084130271232,"[\n {\n ""indices"": [\n 20,\n 26\...","[\n {\n ""id"": 219188946,\n ""id_str"": ""2...",hi,@hanumanbeniwal has been done in all #AIIMS ne...,Negative,anger,"Topic 3: doria (0.09), story (0.05), coronavir..."
1,Sun Apr 12 18:27:41 +0000 2020,1249403835711393792,@mediavakker @vrtnws En waarom zouden deze bur...,1249398547608469511,950489665513062401,54590065,[],"[\n {\n ""id"": 950489665513062401,\n ""id...",nl,@Mediafakker @vrtnws and why would these citiz...,Negative,anger,"Topic 0: would (0.04), nt (0.04), measures (0...."
2,Sun Apr 12 18:27:54 +0000 2020,1249403887871823872,"@_________xxz Mit den Worten ""digital"", ""innov...",1249401882596040705,909103079852822533,1104859974654668800,[],"[\n {\n ""id"": 909103079852822533,\n ""id...",de,"@_________xxz with the words ""digital"", ""innov...",Negative,anger,"Topic 4: ucj4afnh8m (0.04), innovative (0.04),..."
3,Sun Apr 12 18:28:06 +0000 2020,1249403937746227200,Business Game Planning During Corona Virus - h...,None,None,290265283,[],[],en,Business Game Planning During Corona Virus - h...,Neutral,fear,"Topic 1: virus (0.06), wah (0.04), china (0.02..."
4,Sun Apr 12 18:28:17 +0000 2020,1249403986588971012,@HeatingBarrett @Grundfos_UK @gasappuk @CosyWa...,1249285715529261057,3652922657,1319972514,[],"[\n {\n ""id"": 3652922657,\n ""id_str"": ""...",en,@HeatingBarrett @Grundfos_UK @gasappuk @CosyWa...,Positive,anger,"Topic 4: ucj4afnh8m (0.04), innovative (0.04),..."


In [11]:
selected_columns_retweets = [
    'CREATED_AT',
    'ID',
    'TEXT',
    'ORIGINAL_TWEET_ENTITIES_USER_MENTIONS',
    'ORIGINAL_TWEET_ENTITIES_HASHTAGS',
    'ORIGINAL_TWEET_FAVORITE_COUNT',
    'ORIGINAL_TWEET_RETWEET_COUNT',
    'ORIGINAL_TWEET_REPLY_COUNT',
    'ORIGINAL_TWEET_QUOTE_COUNT',
    'ORIGINAL_TWEET_IS_QUOTE_STATUS',
    'ORIGINAL_TWEET_PLACE',
    'ORIGINAL_TWEET_ID',
    'USER_ID',
    'ENTITIES_HASHTAGS',
    'ENTITIES_USER_MENTIONS',
    'ORIGINAL_TWEET_LANG',
    'translated_text',
    'sentiment',
    'emotion',
    'TOPIC_MODELS'
]

# Selecting the columns
df_retweets2 = df_retweets[selected_columns_retweets]

# Display the first few rows of the new DataFrame with the selected columns
df_retweets2.head(20)

,CREATED_AT,ID,TEXT,ORIGINAL_TWEET_ENTITIES_USER_MENTIONS,ORIGINAL_TWEET_ENTITIES_HASHTAGS,ORIGINAL_TWEET_FAVORITE_COUNT,ORIGINAL_TWEET_RETWEET_COUNT,ORIGINAL_TWEET_REPLY_COUNT,ORIGINAL_TWEET_QUOTE_COUNT,ORIGINAL_TWEET_IS_QUOTE_STATUS,ORIGINAL_TWEET_PLACE,ORIGINAL_TWEET_ID,USER_ID,ENTITIES_HASHTAGS,ENTITIES_USER_MENTIONS,ORIGINAL_TWEET_LANG,translated_text,sentiment,emotion,TOPIC_MODELS
0,Sun Apr 12 18:27:32 +0000 2020,1249403797064921089,RT @TheLostPiraTE_: Kapde phen le bhai nhi to ...,[],[],1.0,1.0,1.0,0.0,True,None,1249403057395269633,1220391594295205889,[],"[{""id"":43710217,""id_str"":""43710217"",""indices"":...",hi,Rat @Thalospariratte_,Neutral,joy,"Topic 0: last (0.04), rat (0.04), enter (0.04)..."
1,Sun Apr 12 18:27:39 +0000 2020,1249403826697887746,RT @NetajiBond: 104 out 1075 Corona cases are ...,[],[],1315.0,421.0,26.0,14.0,False,None,1249322804199084033,933915488,[],"[{""id"":1125711161591074816,""id_str"":""112571116...",en,RT @NetajiBond: 104 out 1075 Corona cases are ...,Negative,anger,"Topic 3: cases (0.08), people (0.05), tablighi..."
2,Sun Apr 12 18:27:46 +0000 2020,1249403856246591490,RT @ykidsan___: kalau corona berhenti sebelum ...,[],[],1.0,11.0,4.0,1.0,False,None,1249398948378238976,996985884762300417,[],"[{""id"":1218556914432266240,""id_str"":""121855691...",in,RT @ykidsan___: If Corona stops before I enter...,Neutral,joy,"Topic 0: last (0.04), rat (0.04), enter (0.04)..."
3,Sun Apr 12 18:27:54 +0000 2020,1249403889071411200,RT @caiocopolla: Rodrigo ‘Botafogo’ Maia acaba...,[],[],14983.0,5551.0,532.0,366.0,False,None,1246218744520638471,1898888126,[],"[{""id"":1241211023769718784,""id_str"":""124121102...",pt,RT @caiocoCopolla: Rodrigo ‘Botafogo’ Maia has...,Negative,anger,"Topic 2: hospitals (0.03), fight (0.03), refus..."
4,Sun Apr 12 18:28:02 +0000 2020,1249403923963817988,RT @FigaroRossy: ont dit que biloko yaba chino...,[],[],45.0,55.0,0.0,5.0,False,None,1249379499856330752,937573664467357697,[],"[{""id"":1206390410882494467,""id_str"":""120639041...",fr,Rt @figarorossy: have said that the stuff of t...,Negative,sadness,"Topic 4: last (0.05), one (0.05), said (0.05),..."


In [41]:
df_retweets2.head(20)

,CREATED_AT,ID,TEXT,ORIGINAL_TWEET_ENTITIES_USER_MENTIONS,ORIGINAL_TWEET_ENTITIES_HASHTAGS,ORIGINAL_TWEET_FAVORITE_COUNT,ORIGINAL_TWEET_RETWEET_COUNT,ORIGINAL_TWEET_REPLY_COUNT,ORIGINAL_TWEET_QUOTE_COUNT,ORIGINAL_TWEET_IS_QUOTE_STATUS,ORIGINAL_TWEET_PLACE,ORIGINAL_TWEET_ID,USER_ID,ENTITIES_HASHTAGS,ENTITIES_USER_MENTIONS,ORIGINAL_TWEET_LANG,translated_text,sentiment,emotion,TOPIC_MODELS
0,Sun Apr 12 18:27:32 +0000 2020,1249403797064921089,RT @TheLostPiraTE_: Kapde phen le bhai nhi to ...,[],[],1.0,1.0,1.0,0.0,True,None,1249403057395269633,1220391594295205889,[],"[{""id"":43710217,""id_str"":""43710217"",""indices"":...",hi,Rat @Thalospariratte_,Neutral,joy,"Topic 0: last (0.04), rat (0.04), enter (0.04)..."
1,Sun Apr 12 18:27:39 +0000 2020,1249403826697887746,RT @NetajiBond: 104 out 1075 Corona cases are ...,[],[],1315.0,421.0,26.0,14.0,False,None,1249322804199084033,933915488,[],"[{""id"":1125711161591074816,""id_str"":""112571116...",en,RT @NetajiBond: 104 out 1075 Corona cases are ...,Negative,anger,"Topic 3: cases (0.08), people (0.05), tablighi..."
2,Sun Apr 12 18:27:46 +0000 2020,1249403856246591490,RT @ykidsan___: kalau corona berhenti sebelum ...,[],[],1.0,11.0,4.0,1.0,False,None,1249398948378238976,996985884762300417,[],"[{""id"":1218556914432266240,""id_str"":""121855691...",in,RT @ykidsan___: If Corona stops before I enter...,Neutral,joy,"Topic 0: last (0.04), rat (0.04), enter (0.04)..."
3,Sun Apr 12 18:27:54 +0000 2020,1249403889071411200,RT @caiocopolla: Rodrigo ‘Botafogo’ Maia acaba...,[],[],14983.0,5551.0,532.0,366.0,False,None,1246218744520638471,1898888126,[],"[{""id"":1241211023769718784,""id_str"":""124121102...",pt,RT @caiocoCopolla: Rodrigo ‘Botafogo’ Maia has...,Negative,anger,"Topic 2: hospitals (0.03), fight (0.03), refus..."
4,Sun Apr 12 18:28:02 +0000 2020,1249403923963817988,RT @FigaroRossy: ont dit que biloko yaba chino...,[],[],45.0,55.0,0.0,5.0,False,None,1249379499856330752,937573664467357697,[],"[{""id"":1206390410882494467,""id_str"":""120639041...",fr,Rt @figarorossy: have said that the stuff of t...,Negative,sadness,"Topic 4: last (0.05), one (0.05), said (0.05),..."
5,Sun Apr 12 18:28:09 +0000 2020,1249403952925302784,RT @iAnkurSingh: Single Source opposing lockdo...,[],[],1138.0,882.0,78.0,68.0,False,None,1249391289138728960,48686617,[],"[{""id"":2455275793,""id_str"":""2455275793"",""indic...",en,RT @iAnkurSingh: Single Source opposing lockdo...,Neutral,anger,"Topic 1: appreciate (0.06), ll (0.03), critici..."
6,Sun Apr 12 18:28:16 +0000 2020,1249403981559816192,RT @CNNIndonesia: #DataTerbaruCorona\nAda pena...,[],"[{""indices"":[0,18],""text"":""DataTerbaruCorona""}]",2791.0,2128.0,167.0,325.0,False,None,1249259892319244288,2392125152,"[{""indices"":[18,36],""text"":""DataTerbaruCorona""}]","[{""id"":17128975,""id_str"":""17128975"",""indices"":...",in,RT @CNNIndonesia: #DATERBARUCORONA\nThere are ...,Neutral,joy,"Topic 3: cases (0.08), people (0.05), tablighi..."
7,Sun Apr 12 18:28:23 +0000 2020,1249404012090265601,RT @le_Parisien: [Direct] #coronavirus Les hôp...,[],"[{""indices"":[9,21],""text"":""coronavirus""}]",253.0,176.0,34.0,10.0,False,None,1249217699294728193,800406544776974337,"[{""indices"":[26,38],""text"":""coronavirus""}]","[{""id"":38142665,""id_str"":""38142665"",""indices"":...",fr,RT @le_Parisien: [Direct] #CoronAvirus French ...,Negative,fear,"Topic 2: hospitals (0.03), fight (0.03), refus..."
8,Sun Apr 12 18:28:30 +0000 2020,1249404040892440576,RT @_SJPeace_: Unsung heroes amid Corona pande...,[],[],29631.0,9457.0,337.0,467.0,False,None,1249041637172482051,2239371614,[],"[{""id"":716432228,""id_str"":""716432228"",""indices...",en,RT @_SJPeace_: Unsung heroes amid Corona pande...,Positive,joy,"Topic 1: appreciate (0.06), ll (0.03), critici..."
9,Sun Apr 12 18:28:39 +0000 2020,1249404075889905665,RT @_AyeeCarlos_: THIS MAN IS A GENIUS he figu...,[],[],517026.0,179479.0,1585.0,7962.0,False,"{""attributes"":{},""bounding_box"":{""coordinates""...",1237436114887041024,77

In [13]:
selected_columns_users = [
    'id',
    'id_str',
    'name',
    'screen_name',
    'location',
    'description',
    'followers_count',
    'friends_count',
    'listed_count',
    'favourites_count',
    'statuses_count',
    'created_at',
]

# Selecting the columns
df_users2 = df_users[selected_columns_users]

# Display the first few rows of the new DataFrame with the selected columns
df_users2.head()

,id,id_str,name,screen_name,location,description,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at
0,1242817830946508801,1242817830946508801,juwelz v,juwelz_v,"Lower East Side, Manhattan",Event Lyfe LLC .. Brand Ambassador: #visionary...,43,118,0,722,906,Wed Mar 25 14:17:28 +0000 2020
1,1225145123920588805,1225145123920588805,efe09,efe0927183508,None,Allah'ın en değerli eseri insandır.\nCanı yana...,653,983,0,1255,4177,Wed Feb 05 19:52:38 +0000 2020
2,101007632,101007632,Ravin Gupta,IamRaavin,india,Tweet is personal opinion and Retweet is not e...,499,537,2,4342,4038,Fri Jan 01 16:24:24 +0000 2010
3,1230170166614482944,1230170166614482944,Carpe diem,Carpedi92670638,None,Yezidin hârcı zulüm\nYiğidin burcu ölüm,425,459,0,8830,13101,Wed Feb 19 16:40:09 +0000 2020
4,4707764075,4707764075,UpsidedownTurtle 🧢,Adakisn,None,None,76,82,1,6041,7471,Mon Jan 04 19:43:48 +0000 2016


In [ ]:
import ast
df_tweets2['ENTITIES_HASHTAGS']=df_tweets2['ENTITIES_HASHTAGS'].apply(lambda x: ast.literal_eval(x))


In [24]:
df_tweets2['ENTITIES_USER_MENTIONS']=df_tweets2['ENTITIES_USER_MENTIONS'].apply(lambda x: ast.literal_eval(x))


C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\3514520640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets2['ENTITIES_USER_MENTIONS']=df_tweets2['ENTITIES_USER_MENTIONS'].apply(lambda x: ast.literal_eval(x))


In [34]:
df_tweets2['ENTITIES_HASHTAGS'] = df_tweets2['ENTITIES_HASHTAGS'].apply(lambda x: [item['text'] for item in x] if x else x)
df_tweets2['ENTITIES_HASHTAGS']

C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\3509948573.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets2['ENTITIES_HASHTAGS'] = df_tweets2['ENTITIES_HASHTAGS'].apply(lambda x: [item['text'] for item in x] if x else x)


0        [AIIMS, मेल_फीमेल_Ratio_20_80]
1                                    []
2                                    []
3                                    []
4                                    []
                      ...              
29995                                []
29996                                []
29997                                []
29998                          [Corona]
29999                                []
Name: ENTITIES_HASHTAGS, Length: 30000, dtype: object

In [36]:
df_tweets2['ENTITIES_USER_MENTIONS']=df_tweets2['ENTITIES_USER_MENTIONS'].apply(lambda x: [item['id'] for item in x] if x else x)
df_tweets2['ENTITIES_USER_MENTIONS']

C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\3819878237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets2['ENTITIES_USER_MENTIONS']=df_tweets2['ENTITIES_USER_MENTIONS'].apply(lambda x: [item['id'] for item in x] if x else x)


0                                              [219188946]
1                           [950489665513062401, 23498005]
2                                     [909103079852822533]
3                                                       []
4        [3652922657, 891915230175014912, 7703362978567...
                               ...                        
29995                                                   []
29996                                                   []
29997                                          [185314895]
29998                                          [286042531]
29999                                                   []
Name: ENTITIES_USER_MENTIONS, Length: 30000, dtype: object

['Corona', 'Lockdown21']

In [45]:
df_retweets2['ORIGINAL_TWEET_ENTITIES_HASHTAGS']=df_retweets2['ORIGINAL_TWEET_ENTITIES_HASHTAGS'].apply(lambda x: ast.literal_eval(x) if x else x)
df_retweets2['ORIGINAL_TWEET_ENTITIES_HASHTAGS'] = df_retweets2['ORIGINAL_TWEET_ENTITIES_HASHTAGS'].apply(lambda x: [item['text'] for item in x] if x else x)
df_retweets2['ORIGINAL_TWEET_ENTITIES_HASHTAGS']

C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\2540222958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_retweets2['ORIGINAL_TWEET_ENTITIES_HASHTAGS']=df_retweets2['ORIGINAL_TWEET_ENTITIES_HASHTAGS'].apply(lambda x: ast.literal_eval(x) if x else x)
C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\2540222958.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_retweets2['ORIGINAL_TWEET_ENTITIES_HASHTAGS'] = df_retweets2['ORIGINAL_TWEET_ENTITIES_HASHTAGS'].apply(lambda x: [item['text'] for

0                 []
1                 []
2                 []
3                 []
4                 []
            ...     
9995    [Migrantifa]
9996              []
9997              []
9998              []
9999              []
Name: ORIGINAL_TWEET_ENTITIES_HASHTAGS, Length: 10000, dtype: object

In [49]:
df_retweets2['ORIGINAL_TWEET_ENTITIES_USER_MENTIONS']=df_retweets2['ORIGINAL_TWEET_ENTITIES_USER_MENTIONS'].apply(lambda x: ast.literal_eval(x) if x else x)
df_retweets2['ORIGINAL_TWEET_ENTITIES_USER_MENTIONS']=df_retweets2['ORIGINAL_TWEET_ENTITIES_USER_MENTIONS'].apply(lambda x: [item['id'] for item in x] if x else x)
df_retweets2['ORIGINAL_TWEET_ENTITIES_USER_MENTIONS']

C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\2869702877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_retweets2['ORIGINAL_TWEET_ENTITIES_USER_MENTIONS']=df_retweets2['ORIGINAL_TWEET_ENTITIES_USER_MENTIONS'].apply(lambda x: ast.literal_eval(x) if x else x)
C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\2869702877.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_retweets2['ORIGINAL_TWEET_ENTITIES_USER_MENTIONS']=df_retweets2['ORIGINAL_TWEET_ENTITIES_USER_MENTIONS'].apply(lambda x:

0       []
1       []
2       []
3       []
4       []
        ..
9995    []
9996    []
9997    []
9998    []
9999    []
Name: ORIGINAL_TWEET_ENTITIES_USER_MENTIONS, Length: 10000, dtype: object

In [52]:
df_retweets2['ENTITIES_HASHTAGS']=df_retweets2['ENTITIES_HASHTAGS'].apply(lambda x: ast.literal_eval(x) if x else x)
df_retweets2['ENTITIES_HASHTAGS'] = df_retweets2['ENTITIES_HASHTAGS'].apply(lambda x: [item['text'] for item in x] if x else x)
df_retweets2['ENTITIES_HASHTAGS']

C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\1499453353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_retweets2['ENTITIES_HASHTAGS']=df_retweets2['ENTITIES_HASHTAGS'].apply(lambda x: ast.literal_eval(x) if x else x)
C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\1499453353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_retweets2['ENTITIES_HASHTAGS'] = df_retweets2['ENTITIES_HASHTAGS'].apply(lambda x: [item['text'] for item in x] if x else x)


0                 []
1                 []
2                 []
3                 []
4                 []
            ...     
9995    [Migrantifa]
9996              []
9997              []
9998              []
9999              []
Name: ENTITIES_HASHTAGS, Length: 10000, dtype: object

In [58]:
df_retweets2['ENTITIES_USER_MENTIONS'][0]

'[{"id":43710217,"id_str":"43710217","indices":[3,18],"name":"Mr. Psychopath","screen_name":"TheLostPiraTE_"}]'

In [61]:
import json
df_retweets2['ENTITIES_USER_MENTIONS'] = df_retweets2['ENTITIES_USER_MENTIONS'].apply(lambda x: json.loads(x))
df_retweets2['ENTITIES_USER_MENTIONS'] = df_retweets2['ENTITIES_USER_MENTIONS'].apply(lambda x: [item['id'] for item in x] if x else x)
df_retweets2['ENTITIES_USER_MENTIONS']

C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\2027206356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_retweets2['ENTITIES_USER_MENTIONS'] = df_retweets2['ENTITIES_USER_MENTIONS'].apply(lambda x: json.loads(x))
C:\Users\srinivas\AppData\Local\Temp\ipykernel_13952\2027206356.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_retweets2['ENTITIES_USER_MENTIONS'] = df_retweets2['ENTITIES_USER_MENTIONS'].apply(lambda x: [item['id'] for item in x] if x else x)


0                  [43710217]
1       [1125711161591074816]
2       [1218556914432266240]
3       [1241211023769718784]
4       [1206390410882494467]
                ...          
9995    [1149753717564088322]
9996               [14619744]
9997               [31079332]
9998             [3403949627]
9999             [2719753171]
Name: ENTITIES_USER_MENTIONS, Length: 10000, dtype: object

In [83]:
def unique_hashtags(hashtag_lists):
    # Flatten the list of lists and remove duplicates by converting it to a set
    unique_set = set(hashtag for sublist in hashtag_lists for hashtag in sublist)
    # Convert set back to list for further processing or storage
    return list(unique_set)


df_all_tweets_hashtags = pd.concat([
    df_tweets2[['USER_ID', 'ENTITIES_HASHTAGS']],
    df_retweets2[['USER_ID', 'ENTITIES_HASHTAGS']]
])
# Group by user_id and aggregate all hashtags into one unique list per user
aggregated_hashtags = df_all_tweets_hashtags.groupby('USER_ID')['ENTITIES_HASHTAGS'].agg(unique_hashtags).reset_index()

# Merge this back into df_users



In [84]:
aggregated_hashtags['USER_ID']=aggregated_hashtags['USER_ID'].apply(lambda x: int(x))

In [85]:
df_users2 = pd.merge(df_users2, aggregated_hashtags, left_on='id',right_on='USER_ID', how='left')

# Ensure that users with no tweets or retweets have an empty list instead of NaN
df_users2['ENTITIES_HASHTAGS'] = df_users2['ENTITIES_HASHTAGS'].apply(lambda x: x if isinstance(x, list) else [])

In [86]:
df_users2['ENTITIES_HASHTAGS']

0                               []
1        [COVID19InTurkeysPrisons]
2                           [BSNL]
3        [COVID19InTurkeysPrisons]
4                               []
                   ...            
33941                           []
33942                           []
33943                           []
33944                           []
33945                           []
Name: ENTITIES_HASHTAGS, Length: 33946, dtype: object

In [87]:
def unique_mentions(mention_lists):
    # Flatten the list of lists and remove duplicates by converting it to a set
    unique_set = set(mention for sublist in mention_lists for mention in sublist)
    # Convert set back to list for further processing or storage
    return list(unique_set)


df_all_tweets_mentions = pd.concat([
    df_tweets2[['USER_ID', 'ENTITIES_USER_MENTIONS']],
    df_retweets2[['USER_ID', 'ENTITIES_USER_MENTIONS']]
])
# Group by user_id and aggregate all hashtags into one unique list per user
aggregated_mentions = df_all_tweets_mentions.groupby('USER_ID')['ENTITIES_USER_MENTIONS'].agg(unique_mentions).reset_index()

# Merge this back into df_users



In [88]:
aggregated_mentions['USER_ID']=aggregated_mentions['USER_ID'].apply(lambda x: int(x))

In [89]:
df_users2 = pd.merge(df_users2, aggregated_mentions, left_on='id',right_on='USER_ID', how='left')

# Ensure that users with no tweets or retweets have an empty list instead of NaN


In [90]:
df_users2

,id,id_str,name,screen_name,location,description,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,USER_ID_x,ENTITIES_HASHTAGS,USER_ID_y,ENTITIES_USER_MENTIONS
0,1242817830946508801,1242817830946508801,juwelz v,juwelz_v,"Lower East Side, Manhattan",Event Lyfe LLC .. Brand Ambassador: #visionary...,43,118,0,722,906,Wed Mar 25 14:17:28 +0000 2020,1.242818e+18,[],1.242818e+18,[16144221]
1,1225145123920588805,1225145123920588805,efe09,efe0927183508,None,Allah'ın en değerli eseri insandır.\nCanı yana...,653,983,0,1255,4177,Wed Feb 05 19:52:38 +0000 2020,1.225145e+18,[COVID19InTurkeysPrisons],1.225145e+18,"[1191410595880411138, 1225145123920588805]"
2,101007632,101007632,Ravin Gupta,IamRaavin,india,Tweet is personal opinion and Retweet is not e...,499,537,2,4342,4038,Fri Jan 01 16:24:24 +0000 2010,1.010076e+08,[BSNL],1.010076e+08,"[882219929709105152, 901130659283877889, 84196..."
3,1230170166614482944,1230170166614482944,Carpe diem,Carpedi92670638,None,Yezidin hârcı zulüm\nYiğidin burcu ölüm,425,459,0,8830,13101,Wed Feb 19 16:40:09 +0000 2020,1.230170e+18,[COVID19InTurkeysPrisons],1.230170e+18,"[1113359168159854592, 1242481330102902786]"
4,4707764075,4707764075,UpsidedownTurtle 🧢,Adakisn,None,None,76,82,1,6041,7471,Mon Jan 04 19:43:48 +0000 2016,4.707764e+09,[],4.707764e+09,[14135350]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33941,1078538781303558144,1078538781303558144,samar india,samarindialive2,"Uttar Pradesh, India",None,17,0,0,1,3299,Fri Dec 28 06:30:58 +0000 2018,1.078539e+18,[],1.078539e+18,[]
33942,2864430993,2864430993,Patrick Jansen 🐾,PatrickAJansen,Tweets op persoonlijke titel,Ecoloog | Natuur liefhebber | Pleitbezorger vo...,7516,1991,86,19643,24479,Thu Nov 06 20:47:44 +0000 2014,NaN,[],NaN,NaN
33943,2506192686,2506192686,Dionisius Yosan C,bukanmayoritas,"Malang, Jawa Timur",http://Bolalob.com's Football Writer,71,92,0,70,569,Mon May 19 02:59:34 +0000 2014,NaN,[],NaN,NaN
33944,471094616,471094616,Lexi,AlexiaBoss,"Pretoria, South Africa","Love to travel, experience new places, the foo...",344,1428,5,16530,15559,Sun Jan 22 13:57:12 +0000 2012,4.710946e+08,[],4.710946e+08,[72013267]


In [91]:
selected_columns_users = [
    'id',
    'id_str',
    'name',
    'screen_name',
    'location',
    'description',
    'followers_count',
    'friends_count',
    'listed_count',
    'favourites_count',
    'statuses_count',
    'created_at',
    'ENTITIES_HASHTAGS',
    'ENTITIES_USER_MENTIONS'
]

# Selecting the columns
df_users3 = df_users2[selected_columns_users]

# Display the first few rows of the new DataFrame with the selected columns
df_users3.head()

,id,id_str,name,screen_name,location,description,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,ENTITIES_HASHTAGS,ENTITIES_USER_MENTIONS
0,1242817830946508801,1242817830946508801,juwelz v,juwelz_v,"Lower East Side, Manhattan",Event Lyfe LLC .. Brand Ambassador: #visionary...,43,118,0,722,906,Wed Mar 25 14:17:28 +0000 2020,[],[16144221]
1,1225145123920588805,1225145123920588805,efe09,efe0927183508,None,Allah'ın en değerli eseri insandır.\nCanı yana...,653,983,0,1255,4177,Wed Feb 05 19:52:38 +0000 2020,[COVID19InTurkeysPrisons],"[1191410595880411138, 1225145123920588805]"
2,101007632,101007632,Ravin Gupta,IamRaavin,india,Tweet is personal opinion and Retweet is not e...,499,537,2,4342,4038,Fri Jan 01 16:24:24 +0000 2010,[BSNL],"[882219929709105152, 901130659283877889, 84196..."
3,1230170166614482944,1230170166614482944,Carpe diem,Carpedi92670638,None,Yezidin hârcı zulüm\nYiğidin burcu ölüm,425,459,0,8830,13101,Wed Feb 19 16:40:09 +0000 2020,[COVID19InTurkeysPrisons],"[1113359168159854592, 1242481330102902786]"
4,4707764075,4707764075,UpsidedownTurtle 🧢,Adakisn,None,None,76,82,1,6041,7471,Mon Jan 04 19:43:48 +0000 2016,[],[14135350]


In [92]:
create_schema_sql = """
CREATE SCHEMA PROCESSED_DATA;
"""
session.sql(create_schema_sql).collect()

[Row(status='Schema PROCESSED_DATA successfully created.')]

In [93]:
session.use_schema('PROCESSED_DATA')

In [94]:
session.write_pandas(df_tweets2, 'TWEETS_PROCESSED',auto_create_table=True).collect()
session.write_pandas(df_retweets2, 'RETWEETS_PROCESSED',auto_create_table=True).collect()
session.write_pandas(df_users3, 'USERS_PROCESSED',auto_create_table=True).collect()

[Row(id=1242817830946508801, id_str='1242817830946508801', name='juwelz v', screen_name='juwelz_v', location='Lower East Side, Manhattan', description='Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production🎥📸 #BringNyCultureBack 🌊🌊🌊 !!', followers_count=43, friends_count=118, listed_count=0, favourites_count=722, statuses_count=906, created_at='Wed Mar 25 14:17:28 +0000 2020', ENTITIES_HASHTAGS='[]', ENTITIES_USER_MENTIONS='[\n  16144221\n]'),
 Row(id=1225145123920588805, id_str='1225145123920588805', name='efe09', screen_name='efe0927183508', location=None, description="Allah'ın en değerli eseri insandır.\nCanı yanan sabretsin.\nCan yakan, canının yanacağı günü beklesin..\n677 khk\nRT düşüncem olduğu anlamına gelmez bilgi amaçlıdır", followers_count=653, friends_count=983, listed_count=0, favourites_count=1255, statuses_count=4177, created_at='Wed Feb 05 19:52:38 +0000 2020', ENTITIES_HASHTAGS='[\n  "COVID19InTurkeysPrisons"\n]', 